# Tax-Min-Method

In [119]:
#pip install pandas-datareader

In [120]:
#Import required libraries
import pandas as pd
import numpy as np
from datetime import date
import decimal
#from pandas_datareader import data as pdr
from datetime import datetime

In [121]:
import os
working_directory = os.getcwd()

## Import CSV file and convert "Date Acquired" column to datetime 

In [122]:
path = working_directory + '/Min_Tax_Port.csv'
d_parser = lambda x: datetime.strptime(x, '%m/%d/%Y')
tax_min_port = pd.read_csv(path, parse_dates=["Date Acquired"], date_parser=d_parser)
tax_min_port

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired
0,VXUS,40.0000,65.59,63.6300,78.4000,2021-02-24
1,VXUS,25.0000,65.59,63.1275,61.5625,2021-03-12
2,VXUS,0.1637,65.59,61.6282,0.6486,2021-03-25
3,VXUS,0.5079,65.59,66.0598,-0.2386,2021-06-24
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22
5,VWO,0.0743,51.64,50.6299,0.0750,2021-03-25
6,VWO,0.2858,51.64,54.0164,-0.6793,2021-06-24
7,VUG,10.0000,297.10,259.3800,377.2000,2021-02-24
8,VUG,0.0151,297.10,252.5793,0.6714,2021-03-30
9,VUG,0.0131,297.10,286.3999,0.1397,2021-06-29


# Create variables that gives us the current date and date one year prior.
This logic is to determine the current date and then the date one year prior which would indicate any lots purchased before then (the date one year prior from current date) are definitely long term tax lots.
Some brokerage firms platforms will actually allow you to define which tax lots are long or short, however, that is too easy and defining a function or method to determine this will be helpful for any data that does not identify the tax lots beforehand.

In [123]:
import datetime
#date_now = datetime.datetime.now()
date_now = datetime.date.today()
year_ago = date_now.year - 1

current_date = date_now.strftime('%Y-%m-%d')
one_year_ago = date_now.replace(year=year_ago).strftime('%Y-%m-%d')

print(current_date)
print(one_year_ago)

2021-08-29
2020-08-29


## Alternate dataframe filtering method

In [124]:
#1.) Short-Term losses 2.) Long-Term losses 3.) Long-Term gains 4.) Short-Term gains.
tax_min_port['lt'] = np.where(tax_min_port['Date Acquired'] <= one_year_ago,1,0)
tax_min_port['st_loss'] = np.where((tax_min_port['Total Gain'] < 0) & (tax_min_port['lt']==0),1,0)
tax_min_port['lt_loss'] = np.where((tax_min_port['Total Gain'] < 0) & (tax_min_port['lt']==1),1,0)
tax_min_port['lt_gain'] = np.where((tax_min_port['Total Gain'] > 0) & (tax_min_port['lt']==1),1,0)
tax_min_port['st_gain'] = np.where((tax_min_port['Total Gain'] > 0) & (tax_min_port['lt']==0),1,0)

#confirm no mis-assignments made
assert(tax_min_port.loc[tax_min_port['lt']==1,['st_loss','st_gain']].sum().sum()==0)
assert(tax_min_port.loc[tax_min_port['lt']==0,['lt_loss','lt_gain']].sum().sum()==0)

tax_min_port.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain
0,VXUS,40.0000,65.59,63.6300,78.4000,2021-02-24,0,0,0,0,1
1,VXUS,25.0000,65.59,63.1275,61.5625,2021-03-12,0,0,0,0,1
2,VXUS,0.1637,65.59,61.6282,0.6486,2021-03-25,0,0,0,0,1
3,VXUS,0.5079,65.59,66.0598,-0.2386,2021-06-24,0,1,0,0,0
4,VWO,55.0000,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0


In [125]:
# Add a column that displays the "Total Value" of each lot
tax_min_port["Total Value"] = (tax_min_port["Last Price"] * tax_min_port["Quanity"])

In [126]:
pd.set_option('display.max_rows', 110)

### 1.) Re-organize dataframe in order by categories listed above and display lots in order of biggest loss or lowest gain first:
#### st loss: biggest to smallest
#### lt loss: biggest to smallest
#### lt gain: smallest to biggest
#### st gain: smallest to biggest

In [152]:
tax_min_ordered = tax_min_port.sort_values(by =['st_loss', 'lt_loss', 'lt_gain', 'st_gain', 'Total Gain'], ascending=(False,False,False,False,True))
tax_min_ordered.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
31,TDOC,11.0,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.69
48,PLTR,100.0,22.92,25.9800,-306.0000,2021-01-12,0,1,0,0,0,2292.00
49,PLTR,100.0,22.92,25.0000,-208.0000,2021-02-25,0,1,0,0,0,2292.00
4,VWO,55.0,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.20
50,NIO,25.0,44.22,48.4078,-104.6950,2021-02-23,0,1,0,0,0,1105.50


In [153]:
# Consider a total withdrawal amount to output a proposal of which lots to sell in order
withdrawal_amount = 25000
recommended_tax_lots = tax_min_ordered.loc[tax_min_port['Total Value'].cumsum().le(withdrawal_amount)]
recommended_tax_lots.head()

,Symbol,Quanity,Last Price,Cost/Share,Total Gain,Date Acquired,lt,st_loss,lt_loss,lt_gain,st_gain,Total Value
31,TDOC,11.0,146.79,199.0000,-574.3100,2021-01-05,0,1,0,0,0,1614.69
48,PLTR,100.0,22.92,25.9800,-306.0000,2021-01-12,0,1,0,0,0,2292.00
49,PLTR,100.0,22.92,25.0000,-208.0000,2021-02-25,0,1,0,0,0,2292.00
4,VWO,55.0,51.64,54.9061,-179.6355,2021-02-22,0,1,0,0,0,2840.20
50,NIO,25.0,44.22,48.4078,-104.6950,2021-02-23,0,1,0,0,0,1105.50


In [154]:
# Total value of lots being sold (not accounting for partial sale of next lot)
current_proceeds = recommended_tax_lots['Total Value'].sum()
current_proceeds

24591.673153000003

In [155]:
# Current gain or loss
current_gain_or_loss = recommended_tax_lots["Total Gain"].sum()
current_gain_or_loss

-421.8626999999997

In [157]:
# Difference needed to meet withdrawal total
proceeds_still_needed = withdrawal_amount - current_proceeds
proceeds_still_needed

408.32684699999663

### Determine next lot that will require partial sale to reach total withdrawal amount

In [158]:
#test = recommended_tax_lots.loc[recommended_tax_lots.sort_values('Total Value',ascending=False,ignore_index=True)['Total Value'].cumsum().le(withdrawal_amount)]

In [159]:
# Determine cumalitve sum of dataframe in descedning order
acumsum = np.cumsum(tax_min_port["Total Value"])
acumsum.reset_index().head()

,index,Total Value
0,31,1614.69
1,48,3906.69
2,49,6198.69
3,4,9038.89
4,50,10144.39


In [160]:
# Identify index row that does not exceed withdrawal amount
np.argmax(acumsum > withdrawal_amount) - 1

30

In [161]:
# Identify next tax lot that requires partial sale to reach total withdrawal amount
partial_lot = tax_min_port.iloc[31]
partial_lot

Symbol                           VTV
Quanity                      23.0000
Last Price                  141.1900
Cost/Share                  120.6374
Total Gain                  472.7100
Date Acquired    2020-02-16 00:00:00
lt                                 1
st_loss                            0
lt_loss                            0
lt_gain                            1
st_gain                            0
Total Value                3247.3700
Name: 10, dtype: object

In [162]:
# Number of shares in next lot needed to reach total withdrawal amount
partial_quanity = proceeds_still_needed / partial_lot['Last Price']
partial_quanity

2.8920380126071015

In [163]:
# Total capital gain or loss
current_gain_or_loss + proceeds_still_needed

-13.535853000003044